In [1]:
#from __future__ import division

import time
import numpy as np
import tensorflow as tf

In [2]:
class Model:
    
    def __init__(self):
        
        tf.reset_default_graph()
        
        (x_train_orig, y_train_orig), (x_test_orig, y_test_orig) = tf.keras.datasets.mnist.load_data()
        
        #Flatten the training and test images
        x_train_flat = x_train_orig.reshape(x_train_orig.shape[0], -1)
        x_test_flat = x_test_orig.reshape(x_test_orig.shape[0], -1)
        
        (self.m, self.n_x) = x_train_flat.shape
        self.n_y = 10  
        
        #Normalize image vectors
        self.x_train = x_train_flat / np.float32(255)
        self.x_test = x_test_flat / np.float32(255)        
            
        #Convert training and test labels to on hot matrices
        self.y_train = self.one_hot(y_train_orig, self.n_y)
        self.y_test = self.one_hot(y_test_orig, self.n_y)                     
           
    def one_hot(self, labels, numclasses):
        return np.eye(numclasses)[labels]
    
    #def create_variables(self):
    #    with tf.variable_scope("variables"):
    #        W1 = tf.get_variable("W1", [self.n_x, 500], initializer = tf.contrib.layers.xavier_initializer(seed = 1))
    #        b1 = tf.get_variable("b1", [1, 500], initializer = tf.zeros_initializer())
    #        W2 = tf.get_variable("W2", [500, 500], initializer = tf.contrib.layers.xavier_initializer(seed = 1))
    #        b2 = tf.get_variable("b2", [1, 500], initializer = tf.zeros_initializer())
    #        W3 = tf.get_variable("W3", [500, 10], initializer = tf.contrib.layers.xavier_initializer(seed = 1))
    #        b3 = tf.get_variable("b3", [1, 10], initializer = tf.zeros_initializer()) 
        
    #Test
    def fully_connected(self, A_prev, nbr_inputs, nbr_outputs):
        W = tf.get_variable("W", [nbr_inputs, nbr_outputs], initializer = tf.contrib.layers.xavier_initializer(seed = 1))
        b = tf.get_variable("b", [1, nbr_outputs], initializer = tf.zeros_initializer())            
        Z = tf.add(tf.matmul(A_prev, W), b)
        A = tf.nn.relu(Z)
        return A
        
    def forward_propagation(self, X):
        with tf.variable_scope("layer1", reuse=tf.AUTO_REUSE):
            A1 = self.fully_connected(X, 784, 500)
        with tf.variable_scope("layer2", reuse=tf.AUTO_REUSE):
            A2 = self.fully_connected(A1, 500, 500)
        with tf.variable_scope("layer3", reuse=tf.AUTO_REUSE):
            A3 = self.fully_connected(A2, 500, 10)
            return A3
        
    #def forward_propagation(self, X):
    #    with tf.variable_scope("variables", reuse=True):
    #        W1 = tf.get_variable("W1")
    #        b1 = tf.get_variable("b1")
    #        W2 = tf.get_variable("W2")
    #        b2 = tf.get_variable("b2")
    #        W3 = tf.get_variable("W3")
    #        b3 = tf.get_variable("b3") 
    #    
    #        Z1 = tf.add(tf.matmul(X, W1), b1)
    #        A1 = tf.nn.relu(Z1)
    #        Z2 = tf.add(tf.matmul(A1, W2), b2)
    #        A2 = tf.nn.relu(Z2)
    #        Z3 = tf.add(tf.matmul(A2, W3), b3)
    #        return Z3

    def compute_loss(self, predicted, actual):              
        total_loss = tf.nn.softmax_cross_entropy_with_logits_v2(logits = predicted, labels = actual)
        avg_loss = tf.reduce_mean(total_loss)    
        return avg_loss
    
    def create_optimizer(self, learning_rate):
        optimizer = tf.train.AdamOptimizer(learning_rate)
        return optimizer
    
    def compute_accuracy(self, predicted, actual):
        correct_prediction = tf.equal(tf.argmax(predicted, axis=1), tf.argmax(actual, axis=1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
        return accuracy                 
        
    def train(self, num_epochs = 100):
    
        BATCH_SIZE = 1000
    
        tf.reset_default_graph()
        tf.set_random_seed(1)

        x = tf.placeholder(tf.float32, shape=[None, self.n_x])
        y = tf.placeholder(tf.float32, shape=[None, self.n_y])

        # Define training dataset
        train_dataset = tf.data.Dataset.from_tensor_slices((x, y)).shuffle(self.m).batch(BATCH_SIZE).repeat()        
        train_iterator = train_dataset.make_initializable_iterator()        
        train_features, train_labels = train_iterator.get_next()    
        train_init_op = train_iterator.make_initializer(train_dataset) 

        #self.create_variables()
        
        #Training
        train_predicted = self.forward_propagation(train_features)        
        loss = self.compute_loss(train_predicted, train_labels)        
        optimizer = self.create_optimizer(0.001).minimize(loss)

        #Compute accuracy
        test_predicted = self.forward_propagation(x)                
        accuracy = self.compute_accuracy(test_predicted, y)
        
        init = tf.global_variables_initializer()

        with tf.Session() as sess:
            sess.run(init)
            sess.run(train_init_op, feed_dict = {x : self.x_train, y: self.y_train})
                 
            nbrOfBatches = self.m // BATCH_SIZE

            begin_time = time.time()
            for epoch in range(num_epochs):

                epoch_loss = 0
                
                for _ in range(nbrOfBatches):
                    _, batch_loss = sess.run([optimizer, loss])
                    epoch_loss += batch_loss

                train_accuracy = sess.run(accuracy, feed_dict = {x: self.x_train, y: self.y_train})
                test_accuracy = sess.run(accuracy, feed_dict = {x: self.x_test, y: self.y_test})
                
                if (epoch % 10 == 0):
                    print("epoch: {}, train_loss: {:.4f}, train_accuracy: {:.4f}, test_accuracy: {:.4f}".format(
                        epoch, 
                        epoch_loss / nbrOfBatches, 
                        train_accuracy,
                        test_accuracy
                    ))
                
            end_time = time.time()

            print('Model trained in {:.3f} (hh:mm:ss.ms)'.format(end_time - begin_time))
                     
            train_accuracy = sess.run(accuracy, feed_dict = {x: self.x_train, y: self.y_train})
            print("Accuracy on training set: {:.4f}".format(train_accuracy))
                
            test_accuracy = sess.run(accuracy, feed_dict = {x: self.x_test, y: self.y_test})
            print("Accuracy on test set: {:.4f}".format(test_accuracy))        

In [3]:
model = Model()
model.train()

Instructions for updating:
Colocations handled automatically by placer.

For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

epoch: 0, train_loss: 0.6807, train_accuracy: 0.8583, test_accuracy: 0.8544
epoch: 10, train_loss: 0.2425, train_accuracy: 0.8991, test_accuracy: 0.8836
epoch: 20, train_loss: 0.2316, train_accuracy: 0.8999, test_accuracy: 0.8845
epoch: 30, train_loss: 0.2307, train_accuracy: 0.8999, test_accuracy: 0.8849
epoch: 40, train_loss: 0.2306, train_accuracy: 0.8999, test_accuracy: 0.8852
epoch: 50, train_loss: 0.0231, train_accuracy: 0.9975, test_accuracy: 0.9789
epoch: 60, train_loss: 0.0005, train_accuracy: 0.9999, test_accuracy: 0.9838
epoch: 70, train_loss: 0.0004, train_accuracy: 0.9999, test_accuracy: 0.9836
epoch: 80, train_loss: 0.0003, train_accuracy: 0.9999, test_accuracy: 0.9834
